In [ ]:
# Célula 1: Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, roc_auc_score
from google.colab import files

print("Bibliotecas importadas com sucesso!")

In [ ]:
# Célula 2: Fazer o upload do arquivo bootcamp_train
print("Por favor, faça o upload do arquivo 'bootcamp_train.csv'")
uploaded_train = files.upload()

print("\nUpload do 'bootcamp_train.csv' concluído!")

In [ ]:
# Célula 2.5: Carregar os dados do arquivo para o DataFrame
import io

# Extrai o nome do arquivo do dicionário 'uploaded_train'
train_filename = list(uploaded_train.keys())[0]

# Usa o pandas para ler o arquivo carregado e criar o DataFrame 'df_train'
df_train = pd.read_csv(io.BytesIO(uploaded_train[train_filename]))

print(f"Arquivo '{train_filename}' carregado com sucesso no DataFrame 'df_train'.")

In [ ]:
# Célula 3: Diagnóstico Completo do DataFrame de Treino

print("Executando diagnóstico completo do arquivo de treino...")
print("-" * 50)

# 1. Formato dos dados
print("Formato dos dados:")
print(" - df_train:", df_train.shape)
print()

# 2. Nomes das colunas
print("Colunas encontradas:")
print(list(df_train.columns))
print()

# 3. Tipos de dados (dtypes)
print("Tipos de dados (dtypes) por coluna:")
print(df_train.dtypes.sort_index())
print()

# 4. Separação de colunas numéricas e categóricas
num_cols = df_train.select_dtypes(include=np.number).columns.tolist()
cat_cols = df_train.select_dtypes(exclude=np.number).columns.tolist()
print(f"Colunas Numéricas ({len(num_cols)}):", num_cols)
print(f"Colunas Categóricas ({len(cat_cols)}):", cat_cols)
print()

# 5. Contagem de valores ausentes (NaN)
print("Colunas com mais valores ausentes:")
missing = df_train.isna().sum().sort_values(ascending=False)
print(missing[missing > 0]) # Mostra apenas as colunas que de fato têm valores ausentes
print()

# 6. Verificação de linhas duplicadas
dup = int(df_train.duplicated().sum())
print(f"Linhas duplicadas encontradas: {dup}")
print()

# 7. Estatísticas básicas das variáveis numéricas
print("Estatísticas das variáveis numéricas:")
stats = df_train[num_cols].describe().T[['mean','std','min','max']]
print(stats)
print()

# 8. Cardinalidade das variáveis categóricas (quantos valores únicos)
print("Cardinalidade das variáveis categóricas:")
card = df_train[cat_cols].nunique().sort_values(ascending=False)
print(card)
print()

# 9. Análise das Colunas-Alvo (Targets)
print("Análise das Colunas-Alvo:")
# Adaptamos a lista para as colunas reais do nosso projeto
TARGET_COLS = [
    'falha_maquina',
    'FDF (Falha Desgaste Ferramenta)',
    'FDC (Falha Dissipacao Calor)',
    'FP (Falha Potencia)',
    'FTE (Falha Tensao Excessiva)',
    'FA (Falha Aleatoria)'
]

present_targets = [c for c in TARGET_COLS if c in df_train.columns]

if present_targets:
    print("Alvos encontrados:", present_targets)
    for t in present_targets:
        vc = df_train[t].value_counts(dropna=False)
        total = vc.sum()
        print(f"\n— Distribuição de '{t}':")
        for k, v in vc.items():
            pct = 100.0 * v / total
            print(f"    {str(k):<10}: {v:>6} ({pct:.2f}%)")
else:
    print("Nenhuma das colunas-alvo do projeto foi encontrada.")

print("-" * 50)
print("Diagnóstico concluído.")

Limpeza e Preparação dos Dados (Focado no df_train

In [ ]:
# Célula 4: Limpeza e Preparação dos Dados (Focado no df_train)

print("Iniciando a limpeza dos dados de treino...")

# 1. Limpar colunas de falha (targets)
colunas_para_limpar = [
    'falha_maquina',
    'FDF (Falha Desgaste Ferramenta)',
    'FDC (Falha Dissipacao Calor)',
    'FP (Falha Potencia)',
    'FTE (Falha Tensao Excessiva)',
    'FA (Falha Aleatoria)'
]

mapa_valores = {
    'sim': 1, 'Sim': 1, 'y': 1, '1': 1, 1: 1, True: 1,
    'não': 0, 'Não': 0, 'N': 0, '0': 0, 0: 0, False: 0, 'n': 0
}

# Aplicar a limpeza apenas no df_train
for col in colunas_para_limpar:
    if col in df_train.columns: # Checa se a coluna existe antes de limpar
        df_train[col] = df_train[col].replace(mapa_valores)
        # Converte para numérico, tratando erros e preenchendo o que sobrar com 0
        df_train[col] = pd.to_numeric(df_train[col], errors='coerce').fillna(0).astype(int)

print("\nColunas de falha limpas. Verificando 'falha_maquina':")
print(df_train['falha_maquina'].value_counts())

# 2. Preencher dados faltantes nas features numéricas
numeric_features = ['temperatura_ar', 'temperatura_processo', 'velocidade_rotacional', 'torque', 'desgaste_da_ferramenta']

for col in numeric_features:
    # Preenche valores faltantes com a mediana da coluna
    # Usar a mediana é mais robusto a outliers
    mediana = df_train[col].median()
    df_train[col].fillna(mediana, inplace=True)

print("\nValores nulos preenchidos. Verificando novamente as informações:")
df_train.info()
print("\nLimpeza de dados concluída!")

Análise Exploratória de Dados (EDA) com Dados Limpos

In [ ]:
# Célula 5: Análise Exploratória de Dados (EDA) com Dados Limpos

print("Iniciando a Análise Exploratória de Dados...")
print("-" * 50)


# 1. Análise do Desbalanceamento de Classes
print("\nGráfico 1: Distribuição da 'falha_maquina'")
falha_counts = df_train['falha_maquina'].value_counts()

plt.figure(figsize=(7, 5))
sns.barplot(x=falha_counts.index, y=falha_counts.values, palette='viridis')
plt.title('Distribuição da Falha Geral da Máquina')
plt.ylabel('Contagem de Amostras')
plt.xticks([0, 1], ['Sem Falha', 'Com Falha'])
plt.show()

# 2. Contagem por Tipo Específico de Falha
print("\nGráfico 2: Contagem por Tipo Específico de Falha")
target_cols = [
    'FDF (Falha Desgaste Ferramenta)', 'FDC (Falha Dissipacao Calor)',
    'FP (Falha Potencia)', 'FTE (Falha Tensao Excessiva)', 'FA (Falha Aleatoria)'
]
falhas_especificas = df_train[target_cols].sum().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x=falhas_especificas.index, y=falhas_especificas.values, palette='crest')
plt.title('Contagem de Cada Tipo de Falha Específica')
plt.ylabel('Contagem de Amostras')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


# 3. Distribuição das Variáveis Numéricas
print("\nGráfico 3: Distribuição das Variáveis Numéricas")
# A lista 'numeric_features' foi definida na célula de limpeza
plt.figure(figsize=(15, 10))
df_train[numeric_features].hist(bins=30, color='skyblue', layout=(2, 3), ax=plt.gca())
plt.tight_layout()
plt.show()


# 4. Matriz de Correlação
print("\nGráfico 4: Matriz de Correlação entre Variáveis")
# Vamos correlacionar as features numéricas com a falha geral
cols_for_corr = numeric_features + ['falha_maquina']
correlation_matrix = df_train[cols_for_corr].corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlação')
plt.show()

print("-" * 50)
print("Análise Exploratória concluída.")

Pré-processamento e Divisão dos Dados

In [ ]:
# Célula 6: Pré-processamento e Divisão dos Dados

print("Iniciando o pré-processamento e a divisão dos dados...")
print("-" * 50)


# 1. Definir as colunas de features (X) e de targets (y)
features = ['tipo', 'temperatura_ar', 'temperatura_processo', 'velocidade_rotacional', 'torque', 'desgaste_da_ferramenta']
target_cols = [
    'FDF (Falha Desgaste Ferramenta)', 'FDC (Falha Dissipacao Calor)',
    'FP (Falha Potencia)', 'FTE (Falha Tensao Excessiva)', 'FA (Falha Aleatoria)'
]

X = df_train[features]
y = df_train[target_cols]


# 2. Criar o pipeline de pré-processamento com ColumnTransformer
# Esta ferramenta aplica diferentes transformações para diferentes tipos de colunas.

# Separa as features em dois grupos: numéricas e categóricas
categorical_features = ['tipo']
numerical_features = ['temperatura_ar', 'temperatura_processo', 'velocidade_rotacional', 'torque', 'desgaste_da_ferramenta']

preprocessor = ColumnTransformer(
    transformers=[
        # Etapa 'num': Aplica StandardScaler às colunas numéricas
        ('num', StandardScaler(), numerical_features),
        # Etapa 'cat': Aplica OneHotEncoder à coluna categórica
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])


# 3. Dividir os dados em conjuntos de Treino e Validação
# Usamos 80% para treino e 20% para validação.
# 'stratify' garante que a proporção de falhas seja a mesma em ambos os conjuntos.
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=df_train['falha_maquina']
)

print("Pipeline de pré-processamento criado.")
print("Dados divididos em conjuntos de treino e validação:")
print(f" - Tamanho do conjunto de treino (X_train): {X_train.shape}")
print(f" - Tamanho do conjunto de validação (X_val): {X_val.shape}")

print("-" * 50)
print("Pré-processamento concluído. Os dados estão prontos para o treinamento.")

Treinamento do Modelo

In [ ]:
# Célula 7: Treinamento do Modelo de Machine Learning

print("Iniciando a fase de treinamento do modelo...")
print("-" * 50)


# 1. Definir o classificador base
# Usamos RandomForestClassifier, um modelo robusto para dados tabulares.
# 'class_weight="balanced"' ajuda o modelo a lidar com o desbalanceamento de classes.
base_classifier = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# 2. Definir a estratégia para o problema multirrótulo
# O MultiOutputClassifier treina um classificador separado para cada tipo de falha.
multi_output_model = MultiOutputClassifier(estimator=base_classifier, n_jobs=-1)

# 3. Criar o pipeline final que junta o pré-processamento e o modelo
# Este é o nosso sistema completo e encapsulado.
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', multi_output_model)])

# 4. Treinar o modelo com os dados de treino!
# O comando .fit() inicia o processo de aprendizado.
print("Treinando o modelo com os dados de treino... Isso pode levar alguns instantes.")
model_pipeline.fit(X_train, y_train)

print("-" * 50)
print("Treinamento concluído!")

Avaliação do Modelo

In [ ]:
# Célula 8: Avaliação do Desempenho no Conjunto de Validação

print("Iniciando a avaliação do modelo no conjunto de validação...")
print("-" * 50)


# 1. Fazer previsões nos dados de validação
y_pred_val = model_pipeline.predict(X_val)

# 2. Gerar o Relatório de Classificação para análise de métricas
print("Relatório de Classificação:")
print(classification_report(y_val, y_pred_val, target_names=target_cols, zero_division=0))


# 3. Calcular a métrica AUC-ROC de forma robusta
# (Lida com o caso de falhas extremamente raras)
print("Calculando AUC-ROC Score...")
y_proba_val = model_pipeline.predict_proba(X_val)

probabilities = []
for p in y_proba_val:
    if p.shape[1] == 2:
        probabilities.append(p[:, 1]) # Probabilidade da classe 1 (falha)
    else:
        # Se só há uma coluna, a prob da classe 1 é sempre 0.
        probabilities.append(np.zeros(p.shape[0]))

y_proba_val_reformatted = np.array(probabilities).T

try:
    auc_score = roc_auc_score(y_val, y_proba_val_reformatted, average='weighted')
    print(f"\nAUC-ROC Score (Média Ponderada): {auc_score:.4f}")
except ValueError as e:
    print(f"\nNão foi possível calcular o AUC Score. Motivo: {e}")

print("-" * 50)
print("Avaliação concluída.")

In [ ]:
# Célula 8: Avaliação do Desempenho no Conjunto de Validação
from sklearn.metrics import confusion_matrix

print("Iniciando a avaliação do modelo no conjunto de validação...")
print("-" * 50)


# 1. Fazer previsões nos dados de validação
y_pred_val = model_pipeline.predict(X_val)

# 2. Gerar o Relatório de Classificação para análise de métricas
print("Relatório de Classificação:")
print(classification_report(y_val, y_pred_val, target_names=target_cols, zero_division=0))

# (O código para AUC Score permanece o mesmo, pode ser incluído aqui se desejar)
# ...

# --- NOVO: Visualização com Matrizes de Confusão ---
print("\nGráfico: Matrizes de Confusão por Tipo de Falha")

# Criamos uma figura para organizar os 5 gráficos
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten() # Transforma a grade 2x3 em uma lista de 6 eixos

for i, col in enumerate(target_cols):
    # Calcula a matriz de confusão para cada tipo de falha
    cm = confusion_matrix(y_val[col], y_pred_val[:, i])

    # Define os rótulos para os quadrados da matriz
    labels = [['Verdadeiro Negativo\n'+str(cm[0,0])], ['Falso Positivo\n'+str(cm[0,1])],
              ['Falso Negativo\n'+str(cm[1,0])], ['Verdadeiro Positivo\n'+str(cm[1,1])]]

    # Formata os labels para exibição
    label_text = (np.asarray(labels)).reshape(2,2)

    # Plota a matriz como um heatmap
    sns.heatmap(cm, annot=label_text, fmt='', cmap='Blues', ax=axes[i], cbar=False)

    axes[i].set_title(col)
    axes[i].set_ylabel('Rótulo Verdadeiro')
    axes[i].set_xlabel('Rótulo Previsto')

# Oculta o último eixo que não foi usado
axes[5].axis('off')

plt.tight_layout()
plt.show()

print("-" * 50)
print("Avaliação concluída.")

In [ ]:
# Célula 9: Otimização de Hiperparâmetros com GridSearchCV

from sklearn.model_selection import GridSearchCV

print("Iniciando a otimização de hiperparâmetros com GridSearchCV...")
print("-" * 50)

# 1. Definir o pipeline (o mesmo de antes, sem o treinamento)
# Nosso pipeline já contém o pré-processador e a estrutura do modelo.
base_classifier = RandomForestClassifier(random_state=42, class_weight='balanced')
multi_output_model = MultiOutputClassifier(estimator=base_classifier, n_jobs=-1)
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', multi_output_model)])

# 2. Definir a grade de parâmetros para testar (uma grade "curta")
# Usamos a sintaxe 'nome_do_passo__parametro_do_estimador__parametro_final'
param_grid = {
    'classifier__estimator__n_estimators': [100, 150],       # Número de árvores na floresta
    'classifier__estimator__max_depth': [10, 20],           # Profundidade máxima de cada árvore
    'classifier__estimator__min_samples_leaf': [5, 10]      # Mínimo de amostras em um nó folha
}

# 3. Configurar o GridSearchCV
# 'cv=3' significa que ele usará validação cruzada com 3 folds.
# 'scoring' define a métrica que ele tentará otimizar. 'f1_weighted' é boa para nosso caso.
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=3,
    n_jobs=-1, # Usa todos os processadores disponíveis
    verbose=1  # Mostra o progresso
)

# 4. Executar a busca pelos melhores parâmetros
print("Executando a busca em grade... Isso pode levar vários minutos.")
grid_search.fit(X_train, y_train)

# 5. Exibir os melhores parâmetros encontrados
print("\nMelhores parâmetros encontrados:")
print(grid_search.best_params_)


# 6. Avaliar o MELHOR modelo encontrado pelo GridSearch
print("\n--- Avaliação do Modelo Otimizado no Conjunto de Validação ---")
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_val)

print("\nRelatório de Classificação (Modelo Otimizado):")
print(classification_report(y_val, y_pred_best, target_names=target_cols, zero_division=0))

print("-" * 50)
print("Otimização e avaliação concluídas.")

In [ ]:
# Célula 10: Geração do Arquivo de Submissão Final com o Modelo Otimizado

print("Iniciando a etapa final: Geração do arquivo de submissão...")
print("-" * 50)

# 1. Fazer o upload do arquivo de teste
print("Por favor, faça o upload do arquivo 'bootcamp_test.csv'")
try:
    # Limpa uploads anteriores para evitar confusão
    files.clear()
    uploaded_test = files.upload()
    test_filename = list(uploaded_test.keys())[0]
    df_test = pd.read_csv(io.BytesIO(uploaded_test[test_filename]))
    print(f"\nArquivo '{test_filename}' carregado com sucesso.")

    # 2. Limpar os dados de teste (aplicar as mesmas transformações do treino)
    # É CRUCIAL que o teste passe pela mesma limpeza de features que o treino.
    print("\nLimpando os dados de teste...")
    for col in numeric_features:
        # Preenche valores faltantes com a MEDIANA DO CONJUNTO DE TREINO para evitar data leakage
        mediana = df_train[col].median()
        df_test[col].fillna(mediana, inplace=True)
    print("Limpeza concluída.")


    # 3. Usar o MELHOR modelo encontrado pelo GridSearchCV para prever as PROBABILIDADES
    print("\nUsando o modelo otimizado para gerar as previsões...")
    best_model = grid_search.best_estimator_
    X_test = df_test[features]

    test_probabilities = best_model.predict_proba(X_test)


    # 4. Criar e formatar o DataFrame de submissão
    submission_df = pd.DataFrame()
    submission_df['id'] = df_test['id']

    probabilities_submission = []
    for p in test_probabilities:
        if p.shape[1] == 2:
            probabilities_submission.append(p[:, 1])
        else:
            probabilities_submission.append(np.zeros(p.shape[0]))

    for i, col in enumerate(target_cols):
        submission_df[col] = probabilities_submission[i]


    # 5. Salvar e baixar o arquivo de submissão final
    submission_df.to_csv('submission.csv', index=False)
    print("\nArquivo 'submission.csv' final gerado com sucesso!")
    display(submission_df.head())

    print("\nIniciando o download do arquivo...")
    files.download('submission.csv')
    print("-" * 50)
    print("Processo concluído!")

except Exception as e:
    print(f"\nOcorreu um erro. Certifique-se de que o upload foi concluído. Erro: {e}")